In [1]:
from transformers import TFAutoModelForMaskedLM, DistilBertTokenizerFast

model_base = TFAutoModelForMaskedLM.from_pretrained('distilbert-base-uncased-fine-tuned')
tokenizer_base = DistilBertTokenizerFast.from_pretrained('tokenizer')

model_pre = TFAutoModelForMaskedLM.from_pretrained("distilbert-base-uncased-fine-tuned-cleaned")
tokenizer_pre = DistilBertTokenizerFast.from_pretrained('test-tokenizer-cleaned-30000')


model_large = TFAutoModelForMaskedLM.from_pretrained('larger-data-model/distilbert-base-uncased-fine-tuned')
tokenizer_large = DistilBertTokenizerFast.from_pretrained('larger-data-model/distilbert-tokenizer')

All model checkpoint layers were used when initializing TFDistilBertForMaskedLM.

All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased-fine-tuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.
All model checkpoint layers were used when initializing TFDistilBertForMaskedLM.

All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased-fine-tuned-cleaned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.
All model checkpoint layers were used when initializing TFDistilBertForMaskedLM.

All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at larger-data-model/distilbert-base-uncased-fine-tuned.
If your task is similar 

In [2]:
from transformers import pipeline

mask_filler_base = pipeline(
    "fill-mask", model=model_base, tokenizer=tokenizer_base
)

mask_filler_pre = pipeline(
    "fill-mask", model=model_pre, tokenizer=tokenizer_pre
)

mask_filler_large = pipeline(
    "fill-mask", model=model_large, tokenizer=tokenizer_large
)


In [107]:
# 'ძალიან [MASK] საჭმელია'
# 'ეს პიცა ძალიან [MASK]'
# 'ქართული ენა საკმაოდ [MASK]'
# 'საქართველოს ეროვნული [MASK]'
# 'ბევრი რამ ჯერ კიდევ [MASK] ვიცი'
# 'ჯანსაღი ცხოვრება [MASK] მოსდევს'
# 'ქართული [MASK] სწავლა საკმაოდ რთულია'
# 'გამარჯობა, მე დავითი [MASK]'
# text = 'სტადიონი ინგრეოდა ფანების ძახილით როდესაც რონალდომ [MASK] გაიტანა'
# 'სამაგისტრო ნაშრომისთვის ბევრი [MASK] ფაქტი მჭირდება'

text = 'სტადიონი ინგრეოდა ფანების ძახილით როდესაც რონალდომ [MASK] გაიტანა'


preds = [mask_filler_base(text), mask_filler_pre(text)]
model_names = ['model_base', 'model_pre']

for model, name in zip(preds, model_names):
    print('***** ' + name + ' *****')
    for pred in model:
        print(f">>> {pred['sequence']}", round(pred['score'], 3))
    print("\n")

***** model_base *****
>>> სტადიონი ინგრეოდა ფანების ძახილით როდესაც რონალდომო გაიტანა 0.099
>>> სტადიონი ინგრეოდა ფანების ძახილით როდესაც რონალდომი გაიტანა 0.073
>>> სტადიონი ინგრეოდა ფანების ძახილით როდესაც რონალდომა გაიტანა 0.068
>>> სტადიონი ინგრეოდა ფანების ძახილით როდესაც რონალდომთ გაიტანა 0.064
>>> სტადიონი ინგრეოდა ფანების ძახილით როდესაც რონალდომანის გაიტანა 0.035


***** model_pre *****
>>> სტადიონი ინგრეოდა ფანების ძახილით როდესაც რონალდომ გოლი გაიტანა 0.092
>>> სტადიონი ინგრეოდა ფანების ძახილით როდესაც რონალდომ ვერ გაიტანა 0.055
>>> სტადიონი ინგრეოდა ფანების ძახილით როდესაც რონალდომ გაიტანა გაიტანა 0.05
>>> სტადიონი ინგრეოდა ფანების ძახილით როდესაც რონალდომ ბურთი გაიტანა 0.047
>>> სტადიონი ინგრეოდა ფანების ძახილით როდესაც რონალდომ არ გაიტანა 0.029




In [4]:
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer_pre, mlm_probability=0.15)
wholde_word_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer_pre, mlm_probability=0.15)

In [5]:
from datasets import concatenate_datasets, load_from_disk
lm_datasets = load_from_disk("distilbert-model/lm_dataset-30000")
tf_train_dataset = model_base.prepare_tf_dataset(
    lm_datasets["train"],
    collate_fn=wholde_word_collator,
    shuffle=True,
    batch_size=32,
)

eval_dataset = concatenate_datasets([lm_datasets["test"], lm_datasets["validation"]])
tf_eval_dataset = model_base.prepare_tf_dataset(
    eval_dataset,
    collate_fn=wholde_word_collator,
    shuffle=False,
    batch_size=32,
)

C:\Users\Davit6174\PycharmProjects\pythonProject\venv\lib\site-packages\transformers\data\data_collator.py:951: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(


In [6]:
from transformers import create_optimizer
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model_pre.compile(optimizer=optimizer)
model_base.compile(optimizer=optimizer)

tf.keras.mixed_precision.set_global_policy("mixed_float16")


No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4090, compute capability 8.9


In [9]:
import math

eval_loss = model_pre.evaluate(tf_eval_dataset)
print(f"Perplexity of model_pre: {math.exp(eval_loss):.2f}")

In [10]:
eval_loss = model_base.evaluate(tf_eval_dataset)
print(f"Perplexity of model_base: {math.exp(eval_loss):.2f}")

19662/19662 [==============================] - 2385s 121ms/step - loss: 8.4390
Perplexity of model_base: 4623.76


In [73]:
eval_loss_pre = model_pre.evaluate(tf_eval_dataset)
print(f"Perplexity of model_pre: {math.exp(eval_loss):.2f}")

In [75]:
f = open("perplexity.txt", "w")
f.write(f"Perplexity of model_base: {math.exp(eval_loss):.2f}" + "\n" + f"Perplexity of model_pre: {math.exp(eval_loss_pre):.2f}")
f.close()